In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import spotipy
from spotipy import SpotifyOAuth

from sklearn.feature_extraction.text import CountVectorizer

import settings


In [63]:
# Client ID and Client Secret is called and used
spotify_client_id = settings.SPOTIFY_CLIENT_ID
spotify_client_secret = settings.SPOTIFY_CLIENT_SECRET
spotify_redirect_uri = "http://localhost:5000/callback"

In [64]:
# provide scope of access for Python which requests access from the current user
scope =  'user-library-read user-top-read user-read-private playlist-read-private playlist-read-collaborative playlist-modify-public playlist-modify-private'

# authentication manager ensures secured point of access from spotify to python to client (me)
auth_manager = SpotifyOAuth(
                            scope = scope, 
                            client_id = spotify_client_id,
                            client_secret = spotify_client_secret,
                            redirect_uri = spotify_redirect_uri
                            )

# initial call of spotify using authentication manager
sp = spotipy.Spotify(auth_manager = auth_manager)

In [65]:
liked_songs = []
offset = 0
users_liked = sp.current_user_saved_tracks(limit = 50)

while users_liked["next"] != None:
    liked_songs.extend(users_liked["items"])
    offset+=50
    users_liked = sp.current_user_saved_tracks(offset=offset,limit = 50)

liked_songs.extend(users_liked["items"])

In [66]:
len(liked_songs)

1826

In [67]:
df = pd.DataFrame(list(map(lambda x: x["track"],liked_songs)))

In [68]:
unique_artists = df['artists'].apply(lambda x: x[0]['id']).drop_duplicates().reset_index(drop = True).tolist()

In [24]:
# sp.artists(unique_artists[:50])

In [69]:
len(unique_artists)

718

In [70]:
unique_artists_copy = unique_artists
last = len(unique_artists)%50

artists = [sp.artists(unique_artists[x:x+50])['artists'] if len(unique_artists)-x >= 50 else sp.artists(unique_artists[-last:]) for x in range(0,len(unique_artists),50)]



In [77]:
len(artists)

15

In [87]:
collapse_artist = [item for sublist in artists for item in sublist]
collapse_artist.extend(sp.artists(unique_artists[-last:])['artists'])

In [107]:
collapse_artist_notnull = list(map(lambda x : x if type(x) is dict else None, collapse_artist))
artists = [i for i in collapse_artist_notnull if i is not None]

In [109]:
artists_df = pd.DataFrame(artists)

In [119]:
artists_df.columns

Index(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name',
       'popularity', 'type', 'uri'],
      dtype='object')